# Advanced Python: Building Scalable Applications

### Module 1: Threads, Processes and Coroutines
- Concurrency Vs Parallelism: Choosing generator/coroutines Vs Threads/Processes
- Introduction to threads and processes.
- Python threading module API
   - Creating and managing threads.
   - An overview on threading module.
   - Using the Thread class and the Timer class.
   - Active threads Vs. Daemon threads.
   - Helper functions in the threading module.
- Python multiprocessing module API
   - Multitasking using multiprocessing.Process
   - Process Vs Thread: performance and design implications.
   - Similarities and differences between Process and Thread class API
   - Helper functions in the multiprocessing module.
- Creating thread-pools and process-pools using concurrent.futures package
- Using gevent framework for massive I/O concurrency work-loads
- Implementing co-routines using Python built-in async support
- An overview on uvloop and trio
- Best architectural design practices while choosing between different concurrency models.

#### Generators: how they work

In [1]:
def testfn():
    print("Start of testfn...")
    return 10
    print("Back inside testfn...")

testfn()

Start of testfn...


10

In [ ]:
def testfn():
    print("Start of testfn...")
    yield 10
    print("Back inside testfn...")

g = testfn()  # Returns a generator object. Generators are iterable objects
print(g, type(g))

<generator object testfn at 0x000001F267609150> <class 'generator'>


In [15]:
a = [11, 22, 33, 44, 5]
b = 100
for v in b:
    print(v)

TypeError: 'int' object is not iterable

In [16]:
# This is how python implements the "iterator-protocol" 

a = [11, 22, 33, 44]
b = 100
it = iter(b)
try:
    while True:
        v = next(it)
        # Body of the for loop:
        print(v)
except StopIteration:
    pass


TypeError: 'int' object is not iterable

In [ ]:
range(10)


In [6]:
it = iter(a)
it

In [12]:
next(it)

StopIteration: 

In [ ]:
def testfn():
    print("Start of testfn...")
    v = 100
    yield 10
    print("Back inside testfn...")
    yield "hello"
    print("Back again inside testfn...")
    yield v
    print("End of testfn...")

g = testfn()  # Returns a generator object. Generators are iterable objects
print(g, type(g))

for v in g:
    print("In for loop: v =", v)

<generator object testfn at 0x000001F2690BCF90> <class 'generator'>
Start of testfn...
In for loop: v = 10
Back inside testfn...
In for loop: v = hello
Back again inside testfn...
In for loop: v = None
End of testfn...


In [23]:
testfn.__code__.co_code

b'\x81\x00t\x00d\x01\x83\x01\x01\x00d\x02V\x00\x01\x00t\x00d\x03\x83\x01\x01\x00d\x04V\x00\x01\x00t\x00d\x05\x83\x01\x01\x00d\x00V\x00\x01\x00t\x00d\x06\x83\x01\x01\x00d\x00S\x00'

In [24]:
a = 10
b = 20
c = a + b
c = a.__add__(b)
c = int.__add__(a, b)
c

30

In [ ]:
testfn()
testfn.__call__()

<function __main__.testfn()>

In [ ]:
a = 12345
a = int(12345)
id(a)



2140656321168

In [32]:
a = 1234
b = 1234
print(a, b)
print(id(a), id(b))

1234 1234
2140656323216 2140656323248


In [ ]:
a = 10
b = 10
print(a, b)
print(id(a), id(b))
a += 1 # a = a + 1
print(a, b)
print(id(a), id(b))


10 10
2140545679888 2140545679888
11 10
2140545679920 2140545679888


In [38]:
def fib(x):
    a, b = 0, 1
    for _ in range(x):
        print(a, end=" ")
        a, b = b, a + b

fib(10)

0 1 1 2 3 5 8 13 21 34 

In [43]:
from time import sleep

def fib_list(x):
    series = [0, 1]
    for _ in range(x-2):
        series.append(series[-1] + series[-2])
        sleep(1)
    return series

for v in fib_list(20):
    print(v, v*v)

0 0
1 1
1 1
2 4
3 9
5 25
8 64
13 169
21 441
34 1156
55 3025
89 7921
144 20736
233 54289
377 142129
610 372100
987 974169
1597 2550409
2584 6677056
4181 17480761


In [44]:
from time import sleep

def fib_gen(x):
    a, b = 0, 1
    for _ in range(x):
        yield a
        sleep(1)
        a, b = b, a + b

for v in fib_gen(20):
    print(v, v*v)

0 0
1 1
1 1
2 4
3 9
5 25
8 64
13 169
21 441
34 1156
55 3025
89 7921
144 20736
233 54289
377 142129
610 372100
987 974169
1597 2550409
2584 6677056
4181 17480761


In [48]:
# %load sequential_execution.py
from time import sleep

def foo():
    for i in range(10):
        print(f"foo: counting {i}")
        sleep(1)

def bar():
    for i in range(10):
        print(f"bar: counting {i}")
        sleep(1)

if __name__ == '__main__':
    foo()
    bar()
    
        

foo: counting 0
foo: counting 1
foo: counting 2
foo: counting 3
foo: counting 4
foo: counting 5
foo: counting 6
foo: counting 7
foo: counting 8
foo: counting 9
bar: counting 0
bar: counting 1
bar: counting 2
bar: counting 3
bar: counting 4
bar: counting 5
bar: counting 6
bar: counting 7
bar: counting 8
bar: counting 9


In [54]:
# %load concurrent_execution_using_generators.py
from time import sleep

def foo():
    for i in range(15):
        print(f"foo: counting {i}")
        yield
        sleep(1)
        

def bar():
    for i in range(5):
        print(f"bar: counting {i}")
        yield
        sleep(1)

if __name__ == '__main__':
    g1 = foo()
    g2 = bar()
    print(g1, g2)
    from itertools import zip_longest
    for _ in zip_longest(g1, g2):
        pass

    
        

<generator object foo at 0x000001F26935C9E0> <generator object bar at 0x000001F26935C900>
foo: counting 0
bar: counting 0
foo: counting 1
bar: counting 1
foo: counting 2
bar: counting 2
foo: counting 3
bar: counting 3
foo: counting 4
bar: counting 4
foo: counting 5
foo: counting 6
foo: counting 7
foo: counting 8
foo: counting 9
foo: counting 10
foo: counting 11
foo: counting 12
foo: counting 13
foo: counting 14


In [56]:
# %load concurrent_execution_using_asyncio.py
import asyncio

async def foo():
    for i in range(10):
        print(f"foo: counting {i}")
        await asyncio.sleep(1)

async def bar():
    for i in range(10):
        print(f"bar: counting {i}")
        await asyncio.sleep(1)

async def main():
    await asyncio.gather(foo(), bar())

if __name__ == '__main__':
    asyncio.run(main())

    
        

RuntimeError: asyncio.run() cannot be called from a running event loop

#### Introduction to Threads in Python

Threads are streams of execution pipelines within a running process

Threads can be broadly classified into the following categories:
  1. 1:1 Threading (Native Threads / OS-level Threads / Kernel Supported Threading / Light-Weight-Process). OS takes care of context-switching and scheduling of threads. Threads can be preempted by the OS to switch context other threads waiting for CPU.
  
  2. N:1 Threading (Green Threads / User-level Threads / User Threads / Greenlets / Eventlets / Tasklets / Coroutines). These are threads of execution managed within a process/application without OS intervention for scheduling. There is NO support for preemptive multitasking. All threads in this model are cooperative by nature.
   
  3. M:N Threading (Thread pool architecture)
    

In [57]:
import threading # Provides Native threading support

In [58]:
threading.Thread?

Init signature:
threading.Thread(
    group=None,
    target=None,
    name=None,
    args=(),
    kwargs=None,
    *,
    daemon=None,
)
Docstring:     
A class that represents a thread of control.

This class can be safely subclassed in a limited fashion. There are two ways
to specify the activity: by passing a callable object to the constructor, or
by overriding the run() method in a subclass.
Init docstring:
This constructor should always be called with keyword arguments. Arguments are:

*group* should be None; reserved for future extension when a ThreadGroup
class is implemented.

*target* is the callable object to be invoked by the run()
method. Defaults to None, meaning nothing is called.

*name* is the thread name. By default, a unique name is constructed of
the form "Thread-N" where N is a small decimal number.

*args* is the argument tuple for the target invocation. Defaults to ().

*kwargs* is a dictionary of keyword arguments for the target
invocation. Defaults to {}.

If a